### To view the tensorboard: 
    1: tensorboard --logdir ray_results 
    2: see http://localhost:6006/ in browser

In [2]:
import ray
import ray.rllib.agents.ppo as ppo
from ray.tune.logger import pretty_print
from ray import tune

## 0: RLlib Training APIs: 
1: At a high level, RLlib provides an Trainer class which holds a policy for environment interaction. Through the trainer interface, the policy can be trained, checkpointed, or an action computed. In multi-agent training, the trainer manages the querying and optimization of multiple policies at once.

2: rllib train --run DQN --env CartPole-v0  --config '{"num_workers": 8}'
    To see the tensorboard: tensorboard --logdir=~/ray_results

3: rllib rollout ~/ray_results/default/DQN_CartPole-v0_0upjmdgr0/checkpoint_1/checkpoint-1 \
    --run DQN --env CartPole-v0 --steps 10000

4: Loading and restoring a trained agent from a checkpoint is simple:
    
    agent = ppo.PPOTrainer(config=config, env=env_class)
    agent.restore(checkpoint_path)
    
5: Computing Actions

The simplest way to programmatically compute actions from a trained agent is to use trainer.compute_action(). This method preprocesses and filters the observation before passing it to the agent policy. Here is a simple example of testing a trained agent for one episode:

    # instantiate env class
    env = env_class(env_config)

    # run until episode ends
    episode_reward = 0
    done = False
    obs = env.reset()
    while not done:
        action = agent.compute_action(obs)
        obs, reward, done, info = env.step(action)
        episode_reward += reward
6: It’s recommended that you run RLlib trainers with Tune, for easy experiment management and visualization of results. Just set "run": ALG_NAME, "env": ENV_NAME in the experiment . config. All RLlib trainers are compatible with the Tune API. This enables them to be easily used in experiments with Tune/

7: tune.run() returns an ExperimentAnalysis object that allows further analysis of the training results and retrieving the checkpoint(s) of the trained agent. It also simplifies saving the trained agent. For example:

a: tune.run() allows setting a custom log directory (other than ``~/ray-results``) and automatically saving the trained agent

    analysis = ray.tune.run(
        ppo.PPOTrainer,
        config=config,
        local_dir=log_dir,
        stop=stop_criteria,
        checkpoint_at_end=True)

b: list of lists: one list per checkpoint; each checkpoint list contains 1st the path, 2nd the metric value

        checkpoints = analysis.get_trial_checkpoints_paths(
            trial=analysis.get_best_trial("episode_reward_mean"),
            metric="episode_reward_mean")

c: or simply get the last checkpoint (with highest "training_iteration")

        last_checkpoint = analysis.get_last_checkpoint()
    
d: if there are multiple trials, select a specific trial or automatically choose the best one according to a given metric

        last_checkpoint = analysis.get_last_checkpoint(
            metric="episode_reward_mean", mode="max"
        )

e: Loading and restoring a trained agent from a checkpoint is simple:

    agent = ppo.PPOTrainer(config=config, env=env_class)
    agent.restore(checkpoint_path)

In [2]:
ray.shutdown()
ray.init()

2021-10-16 02:47:55,173	INFO services.py:1252 -- View the Ray dashboard at http://127.0.0.1:8267


{'node_ip_address': '172.16.21.53',
 'raylet_ip_address': '172.16.21.53',
 'redis_address': '172.16.21.53:48500',
 'object_store_address': '/tmp/ray/session_2021-10-16_02-47-53_348974_46171/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-10-16_02-47-53_348974_46171/sockets/raylet',
 'webui_url': '127.0.0.1:8267',
 'session_dir': '/tmp/ray/session_2021-10-16_02-47-53_348974_46171',
 'metrics_export_port': 64532,
 'node_id': 'fc79172c023b86b61972689708962114fb1c722f304004b0b5893f8a'}

#### 1 Example of Traing a PPO Agent

In [5]:
config = ppo.DEFAULT_CONFIG.copy()
config['num_gpus'] = 1
config['num_workers'] = 2
trainer = ppo.PPOTrainer(config = config, env='CartPole-v1') 

for i in range(30):
    result = trainer.train()
    if i % 10 ==0:
        checkpoint = trainer.save()
        print('checkpoint saved at:', checkpoint) 

(pid=48648) WARNING:tensorflow:From /home/ec2-user/anaconda3/envs/tensorflow_p36/gpu_cuda10.0/lib/python3.6/site-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
(pid=48648) 
(pid=48647) WARNING:tensorflow:From /home/ec2-user/anaconda3/envs/tensorflow_p36/gpu_cuda10.0/lib/python3.6/site-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
(pid=48647) 
(pid=48648) 2021-10-16 16:08:08.679896: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
(pid=48648) WARNING:tensorflow:From /home/ec2-user/anaconda3/envs/tensorflow_p36/gpu_cuda10.0/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated a

(RolloutWorker pid=48648) [2021-10-16 16:08:09.558 ip-172-16-19-112:48648 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
(RolloutWorker pid=48648) [2021-10-16 16:08:09.592 ip-172-16-19-112:48648 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
(RolloutWorker pid=48647) [2021-10-16 16:08:09.611 ip-172-16-19-112:48647 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
(RolloutWorker pid=48647) [2021-10-16 16:08:09.646 ip-172-16-19-112:48647 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


2021-10-16 16:08:13,441	WARNING trainer_template.py:186 -- `execution_plan` functions should accept `trainer`, `workers`, and `config` as args!


checkpoint saved at: /home/ec2-user/ray_results/PPO_CartPole-v1_2021-10-16_16-08-04wkiy6cxa/checkpoint_000001/checkpoint-1
checkpoint saved at: /home/ec2-user/ray_results/PPO_CartPole-v1_2021-10-16_16-08-04wkiy6cxa/checkpoint_000011/checkpoint-11
checkpoint saved at: /home/ec2-user/ray_results/PPO_CartPole-v1_2021-10-16_16-08-04wkiy6cxa/checkpoint_000021/checkpoint-21


#### 2 Example of Using Tune

In [4]:
alg = 'PPO'
tune.run(alg,
    stop={'episode_reward_mean':200},
    config={
        'env':'CartPole-v0',
        'num_gpus':1,
        'num_workers':2,
        'lr':tune.grid_search([.01,.001,.0001])     
    }
)

Trial name,status,loc,lr
PPO_CartPole-v0_eda9d_00000,PENDING,,0.01
PPO_CartPole-v0_eda9d_00001,PENDING,,0.001
PPO_CartPole-v0_eda9d_00002,PENDING,,0.0001


(pid=41261) WARNING:tensorflow:From /home/ec2-user/anaconda3/envs/tensorflow_p36/gpu_cuda10.0/lib/python3.6/site-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
(pid=41261) 
(pid=41259) WARNING:tensorflow:From /home/ec2-user/anaconda3/envs/tensorflow_p36/gpu_cuda10.0/lib/python3.6/site-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
(pid=41259) 
(pid=41260) WARNING:tensorflow:From /home/ec2-user/anaconda3/envs/tensorflow_p36/gpu_cuda10.0/lib/python3.6/site-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
(pid=41260) 
(pid=41259) 2021-10-16 15:44:24,968	INFO trainer.py:741 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=41259) 2021-10-16 15:44:24,968	INFO ppo.py:165 -- In multi-agent mo

Trial name,status,loc,lr
PPO_CartPole-v0_eda9d_00000,RUNNING,,0.01
PPO_CartPole-v0_eda9d_00001,RUNNING,,0.001
PPO_CartPole-v0_eda9d_00002,RUNNING,,0.0001


(pid=41366) WARNING:tensorflow:From /home/ec2-user/anaconda3/envs/tensorflow_p36/gpu_cuda10.0/lib/python3.6/site-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
(pid=41366) 
(pid=41367) WARNING:tensorflow:From /home/ec2-user/anaconda3/envs/tensorflow_p36/gpu_cuda10.0/lib/python3.6/site-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
(pid=41367) 
(pid=41370) WARNING:tensorflow:From /home/ec2-user/anaconda3/envs/tensorflow_p36/gpu_cuda10.0/lib/python3.6/site-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
(pid=41370) 
(pid=41373) WARNING:tensorflow:From /home/ec2-user/anaconda3/envs/tensorflow_p36/gpu_cuda10.0/lib/python3.6/site-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use 

Trial name,status,loc,lr
PPO_CartPole-v0_eda9d_00000,RUNNING,,0.01
PPO_CartPole-v0_eda9d_00001,RUNNING,,0.001
PPO_CartPole-v0_eda9d_00002,RUNNING,,0.0001


...

Result for PPO_CartPole-v0_eda9d_00000:
  agent_timesteps_total: 72000
  custom_metrics: {}
  date: 2021-10-16_15-47-01
  done: true
  episode_len_mean: 200.0
  episode_media: {}
  episode_reward_max: 200.0
  episode_reward_mean: 200.0
  episode_reward_min: 200.0
  episodes_this_iter: 20
  episodes_total: 564
  experiment_id: fee4be6e696a4767b3e6c9df50a77f99
  hostname: ip-172-16-19-112
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 0.009999999776482582
          entropy: 0.1091565415263176
          entropy_coeff: 0.0
          kl: 0.07244950532913208
          model: {}
          policy_loss: 0.005926945246756077
          total_loss: 325.9423522949219
          vf_explained_var: 0.4758451282978058
          vf_loss: 325.92193603515625
        train: null
    num_agent_steps_sampled: 72000
    num_agent_steps_trained: 72000
    num_steps_sampled: 72000
    num_steps_trained: 72000
  iterations_since_rest

Result for PPO_CartPole-v0_eda9d_00002:
  agent_timesteps_total: 76000
  custom_metrics: {}
  date: 2021-10-16_15-47-07
  done: false
  episode_len_mean: 199.95
  episode_media: {}
  episode_reward_max: 200.0
  episode_reward_mean: 199.95
  episode_reward_min: 195.0
  episodes_this_iter: 20
  episodes_total: 632
  experiment_id: 36a8061aa2d24b2a90c2e8a7584a9cbe
  hostname: ip-172-16-19-112
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 9.999999747378752e-05
          entropy: 0.4171346127986908
          entropy_coeff: 0.0
          kl: 0.0031355975661426783
          model: {}
          policy_loss: 0.0022702447604388
          total_loss: 273.60052490234375
          vf_explained_var: 0.536511242389679
          vf_loss: 273.5976257324219
        train: null
    num_agent_steps_sampled: 76000
    num_agent_steps_trained: 76000
    num_steps_sampled: 76000
    num_steps_trained: 76000
  iterations_since_r

Result for PPO_CartPole-v0_eda9d_00002:
  agent_timesteps_total: 80000
  custom_metrics: {}
  date: 2021-10-16_15-47-15
  done: true
  episode_len_mean: 200.0
  episode_media: {}
  episode_reward_max: 200.0
  episode_reward_mean: 200.0
  episode_reward_min: 200.0
  episodes_this_iter: 20
  episodes_total: 652
  experiment_id: 36a8061aa2d24b2a90c2e8a7584a9cbe
  hostname: ip-172-16-19-112
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 9.999999747378752e-05
          entropy: 0.39377790689468384
          entropy_coeff: 0.0
          kl: 0.002887232694774866
          model: {}
          policy_loss: -0.0012949386145919561
          total_loss: 222.5024871826172
          vf_explained_var: 0.6581946015357971
          vf_loss: 222.503173828125
        train: null
    num_agent_steps_sampled: 80000
    num_agent_steps_trained: 80000
    num_steps_sampled: 80000
    num_steps_trained: 80000
  iterations_since_r

(pid=41370) [2021-10-16 15:47:15,698 E 41370 41370] raylet_client.cc:159: IOError: Broken pipe [RayletClient] Failed to disconnect from raylet.
(pid=41373) 2021-10-16 15:47:15,697	ERROR worker.py:428 -- SystemExit was raised from the worker
(pid=41373) Traceback (most recent call last):
(pid=41373)   File "python/ray/_raylet.pyx", line 561, in ray._raylet.execute_task
(pid=41373)   File "python/ray/_raylet.pyx", line 568, in ray._raylet.execute_task
(pid=41373)   File "python/ray/_raylet.pyx", line 572, in ray._raylet.execute_task
(pid=41373)   File "python/ray/_raylet.pyx", line 522, in ray._raylet.execute_task.function_executor
(pid=41373)   File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ray/_private/function_manager.py", line 579, in actor_method_executor
(pid=41373)     return method(__ray_actor, *args, **kwargs)
(pid=41373)   File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ray/util/tracing/tracing_helper.py", line 449,

TuneError: ('Trials did not complete', [PPO_CartPole-v0_eda9d_00001])

In [4]:
alg = 'DDPG'
tune.run(alg,
    stop={"training_iteration": 30},
    config={
        'env':'Pendulum-v0',
        'num_gpus':0,
        'num_workers':2,
        'lr':tune.grid_search([.001,])     
    }
)


Trial name,status,loc,lr
DDPG_Pendulum-v0_6b1dd_00000,RUNNING,,0.001


(pid=17553) WARNING:tensorflow:From /Users/mingjunwang/opt/miniconda3/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=17553) Instructions for updating:
(pid=17553) non-resource variables are not supported in the long term
(pid=17553) 2021-07-13 20:54:31,552	INFO trainer.py:591 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=17553) 2021-07-13 20:54:31,552	INFO trainer.py:616 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=17556) WARNING:tensorflow:From /Users/mingjunwang/opt/miniconda3/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=17556) Instructions for updating:
(pid=17556

Result for DDPG_Pendulum-v0_6b1dd_00000:
  custom_metrics: {}
  date: 2021-07-13_20-54-41
  done: false
  episode_len_mean: 200.0
  episode_reward_max: -781.1207639597676
  episode_reward_mean: -1131.8902231793475
  episode_reward_min: -1410.9375467664963
  episodes_this_iter: 6
  episodes_total: 6
  experiment_id: 43107a19371e49529eb221515032f0aa
  hostname: Mingjuns-MacBook-Pro.local
  info:
    last_target_update_ts: 1500
    learner:
      default_policy:
        max_q: 0.1796478033065796
        mean_q: -0.10519210994243622
        min_q: -0.6069953441619873
        model: {}
    num_steps_sampled: 1500
    num_steps_trained: 256
    num_target_updates: 1
  iterations_since_restore: 1
  node_ip: 192.168.0.23
  num_healthy_workers: 2
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 25.799999999999997
    ram_util_percent: 63.55
  pid: 17553
  policy_reward_max: {}
  policy_reward_mean: {}
  policy_reward_min: {}
  sampler_perf:
    mean_action_processing_ms: 0.0973585918644

Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DDPG_Pendulum-v0_6b1dd_00000,RUNNING,192.168.0.23:17553,0.001,1,2.53136,1500,-1131.89,-781.121,-1410.94,200


Result for DDPG_Pendulum-v0_6b1dd_00000:
  custom_metrics: {}
  date: 2021-07-13_20-54-58
  done: false
  episode_len_mean: 200.0
  episode_reward_max: -781.1207639597676
  episode_reward_mean: -1331.3202113249129
  episode_reward_min: -1796.9534032278605
  episodes_this_iter: 6
  episodes_total: 12
  experiment_id: 43107a19371e49529eb221515032f0aa
  hostname: Mingjuns-MacBook-Pro.local
  info:
    last_target_update_ts: 2500
    learner:
      default_policy:
        max_q: -0.50157630443573
        mean_q: -11.473435401916504
        min_q: -24.651611328125
        model: {}
    num_steps_sampled: 2500
    num_steps_trained: 128256
    num_target_updates: 501
  iterations_since_restore: 2
  node_ip: 192.168.0.23
  num_healthy_workers: 2
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 23.545833333333334
    ram_util_percent: 63.48333333333334
  pid: 17553
  policy_reward_max: {}
  policy_reward_mean: {}
  policy_reward_min: {}
  sampler_perf:
    mean_action_processing_ms: 0.

...

Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DDPG_Pendulum-v0_6b1dd_00000,RUNNING,192.168.0.23:17553,0.001,29,507.24,29500,-461.236,-1.11165,-1519.85,200


Result for DDPG_Pendulum-v0_6b1dd_00000:
  custom_metrics: {}
  date: 2021-07-13_21-03-25
  done: true
  episode_len_mean: 200.0
  episode_reward_max: -1.1116457547643066
  episode_reward_mean: -444.3119269581459
  episode_reward_min: -1524.5879513507703
  episodes_this_iter: 6
  episodes_total: 152
  experiment_id: 43107a19371e49529eb221515032f0aa
  hostname: Mingjuns-MacBook-Pro.local
  info:
    last_target_update_ts: 30500
    learner:
      default_policy:
        max_q: 5.001977920532227
        mean_q: -80.31802368164062
        min_q: -147.42247009277344
        model: {}
    num_steps_sampled: 30500
    num_steps_trained: 3712256
    num_target_updates: 14501
  iterations_since_restore: 30
  node_ip: 192.168.0.23
  num_healthy_workers: 2
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 25.751851851851853
    ram_util_percent: 63.059259259259264
  pid: 17553
  policy_reward_max: {}
  policy_reward_mean: {}
  policy_reward_min: {}
  sampler_perf:
    mean_action_processi

Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DDPG_Pendulum-v0_6b1dd_00000,TERMINATED,,0.001,30,526.471,30500,-444.312,-1.11165,-1524.59,200


Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DDPG_Pendulum-v0_6b1dd_00000,TERMINATED,,0.001,30,526.471,30500,-444.312,-1.11165,-1524.59,200


2021-07-13 21:03:26,318	INFO tune.py:448 -- Total run time: 540.24 seconds (539.55 seconds for the tuning loop).


## 1: RLlib Environments

1: RLlib works with several different types of environments, including OpenAI Gym, user-defined, multi-agent, and also batched environments.

2: RLlib uses Gym as its environment interface for single-agent training.



#### 1: Configuring Environments

    https://github.com/ray-project/ray/blob/master/rllib/examples/custom_env.py

In [3]:
import gym, ray
from ray.rllib.agents import ppo

class MyEnv(gym.Env):
    def __init__(self, env_config):
        self.action_space = <gym.Space>
        self.observation_space = <gym.Space>
    def reset(self):
        return <obs>
    def step(self, action):
        return <obs>, <reward: float>, <done: bool>, <info: dict>

ray.init()
trainer = ppo.PPOTrainer(env=MyEnv, config={
    "env_config": {},  # config to pass to env class
})

while True:
    print(trainer.train())

SyntaxError: invalid syntax (<ipython-input-3-7352217a18a1>, line 6)